# MMAI 891 - Topic Analysis

The goal of this project is to build a text classification model by using NLP techniques

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!pip install -q transformers tfds-nightly

     |████████████████████████████████| 3.8 MB 8.2 MB/s 
     |████████████████████████████████| 4.3 MB 46.3 MB/s 
     |████████████████████████████████| 6.5 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
     |████████████████████████████████| 895 kB 54.1 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline


import matplotlib.pyplot as plt
import tensorflow.keras as keras
import pandas as pd
import re
import unicodedata
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

try: 
    from transformers import DistilBertTokenizer, TFDistilBertModel
except Exception as err: # so we catch the error and import it again
    from transformers import DistilBertTokenizer, TFDistilBertModel
from keras.layers import Activation
import numpy as np
import tensorflow as tf
from keras import utils
from keras.models import Sequential
from keras.preprocessing import text, sequence
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, GlobalMaxPool1D, LSTM, TimeDistributed, GRU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow_datasets as tfds
from tensorflow.keras import regularizers

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Data Preparation

In [5]:
train = pd.read_csv("/content/drive/MyDrive/NLP - Project/April 22/public_data.csv")
test=pd.read_csv("/content/drive/MyDrive/NLP - Project/April 22/input_data.csv")
print(train.info())
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10466 entries, 0 to 10465
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10466 non-null  int64 
 1   message  10466 non-null  object
 2   label    10466 non-null  object
dtypes: int64(1), object(2)
memory usage: 245.4+ KB
None
     id                                            message  \
0  5396  I made a purchase recently but I have decided ...   
1  2202                        Cancel my transfer, please.   
2  3768                      How do I change my last name?   
3  4967  A withdraw I tried to make at the ATM was decl...   
4  5622  I tried to buy something online yesterday and ...   

                      label  
0            request_refund  
1           cancel_transfer  
2     edit_personal_details  
3  declined_cash_withdrawal  
4         declined_transfer  


In [6]:
train.head(20)

,id,message,label
0,5396,I made a purchase recently but I have decided ...,request_refund
1,2202,"Cancel my transfer, please.",cancel_transfer
2,3768,How do I change my last name?,edit_personal_details
3,4967,A withdraw I tried to make at the ATM was decl...,declined_cash_withdrawal
4,5622,I tried to buy something online yesterday and ...,declined_transfer
5,7752,is there a reason i need to verify top up,verify_top_up
6,8662,I transferred some funds but I don't see it,balance_not_updated_after_bank_transfer
7,1362,I need to use my disposable card multiple time...,disposable_card_limits
8,6306,A transaction shows duplicate times.,transaction_charged_twice
9,1029,How quickly can you get me my card?,card_delivery_estimate


In [7]:
len(train)

10466

In [8]:
data_reduced = train.groupby('label').filter(lambda x: len(x) > 100)
data_reduced.shape

(10083, 3)

In [9]:
train['label'].unique()

array(['request_refund', 'cancel_transfer', 'edit_personal_details',
       'declined_cash_withdrawal', 'declined_transfer', 'verify_top_up',
       'balance_not_updated_after_bank_transfer',
       'disposable_card_limits', 'transaction_charged_twice',
       'card_delivery_estimate', 'exchange_rate',
       'extra_charge_on_statement', 'terminate_account',
       'top_up_reverted', 'direct_debit_payment_not_recognised',
       'apple_pay_or_google_pay', 'exchange_charge',
       'card_payment_fee_charged', 'age_limit', 'passcode_forgotten',
       'Refund_not_showing_up', 'top_up_by_bank_transfer_charge',
       'why_verify_identity', 'lost_or_stolen_card', 'change_pin',
       'visa_or_mastercard', 'verify_my_identity',
       'verify_source_of_funds', 'cash_withdrawal_not_recognised',
       'transfer_timing', 'supported_cards_and_currencies',
       'unable_to_verify_identity', 'card_arrival',
       'pending_cash_withdrawal', 'beneficiary_not_allowed',
       'lost_or_stolen_phon

In [10]:
len(train['label'].unique())

77

In [11]:
X = train['message']
y = train['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(8372,)
(8372,)
(2094,)
(2094,)


- Are there any duplicate entries? 
- What is the range of lengths for the sentences? Should we impose a minimum sentence length?
- Are there "non-sentence" entries? For example, hashtags or other features we should remove? (luckily, this dataset is quite clean, but that might not always be the case!)

In [12]:
def clean_data(df):
  # clean stop words 
    for s in df:
        print(''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [13]:
clean_data(train)

id
message
label


In [14]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [15]:
def clean_stopwords_shortwords(w):
    stopwords_list=set(stopwords.words('english'))
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

In [16]:
for i in train['message'][1:6]:
    print(f"original: {i}\nfilter: {clean_stopwords_shortwords(i)}")
    print('\n')

original: Cancel my transfer, please.
filter: Cancel transfer, please.


original: How do I change my last name?
filter: How change last name?


original: A withdraw I tried to make at the ATM was declined I don't know why.
filter: withdraw tried make ATM declined know why.


original: I tried to buy something online yesterday and was told declined. I got the same error today after trying again. What is the problem?
filter: tried buy something online yesterday told declined. got error today trying again. What problem?


original: is there a reason i need to verify top up
filter: reason need verify top




In [17]:
l = [clean_stopwords_shortwords(i) for i in train['message'][:11]]
l

["made purchase recently decided I'm buying need receive money back. Can please give refund ASAP. It's extremely urgent.",
 'Cancel transfer, please.',
 'How change last name?',
 'withdraw tried make ATM declined know why.',
 'tried buy something online yesterday told declined. got error today trying again. What problem?',
 'reason need verify top',
 'transferred funds see',
 'need use disposable card multiple times day, cutoff limit?',
 'transaction shows duplicate times.',
 'How quickly get card?',
 'find exchange rates?']

In [18]:
def preprocess_sentence(w):
        #remove extra spaces
        w = unicode_to_ascii(w.lower().strip())
        print(f"0 : {w}")
        # replace ([?.!,¿]) with space
        w = re.sub(r"([?.!,¿])", r" ", w)
        print(f"1 : {w}")
        # replace space
        w = re.sub(r'[" "]+', " ", w)
        print(f"2 : {w}")
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
        print(f"3 : {w}")
        w=clean_stopwords_shortwords(w)
        print(f"4 : {w}")
        w=re.sub(r'@\w+', '',w)
        print(f"5 : {w}")
        return w

In [19]:
for i in l[:3]:
#     print(f"original: {i}\nfilter: {preprocess_sentence(i)}")
#     print('\n')
    preprocess_sentence(i)

0 : made purchase recently decided i'm buying need receive money back. can please give refund asap. it's extremely urgent.
1 : made purchase recently decided i'm buying need receive money back  can please give refund asap  it's extremely urgent 
2 : made purchase recently decided i'm buying need receive money back can please give refund asap it's extremely urgent 
3 : made purchase recently decided i m buying need receive money back can please give refund asap it s extremely urgent 
4 : made purchase recently decided buying need receive money back please give refund asap extremely urgent
5 : made purchase recently decided buying need receive money back please give refund asap extremely urgent
0 : cancel transfer, please.
1 : cancel transfer  please 
2 : cancel transfer please 
3 : cancel transfer please 
4 : cancel transfer please
5 : cancel transfer please
0 : how change last name?
1 : how change last name 
2 : how change last name 
3 : how change last name 
4 : change last name
5 : c

In [20]:
train.head()

,id,message,label
0,5396,I made a purchase recently but I have decided ...,request_refund
1,2202,"Cancel my transfer, please.",cancel_transfer
2,3768,How do I change my last name?,edit_personal_details
3,4967,A withdraw I tried to make at the ATM was decl...,declined_cash_withdrawal
4,5622,I tried to buy something online yesterday and ...,declined_transfer


In [21]:
train['sentence']=train['message'].map(preprocess_sentence)
train.head(10)

Streaming output truncated to the last 5000 lines.
1 : i used an atm in a foreign currency but the rate applied is wrong 
2 : i used an atm in a foreign currency but the rate applied is wrong 
3 : i used an atm in a foreign currency but the rate applied is wrong 
4 : used atm foreign currency rate applied wrong
5 : used atm foreign currency rate applied wrong
0 : can i exchange money from abroad without additional costs?
1 : can i exchange money from abroad without additional costs 
2 : can i exchange money from abroad without additional costs 
3 : can i exchange money from abroad without additional costs 
4 : exchange money abroad without additional costs
5 : exchange money abroad without additional costs
0 : if i need delivery on a certain day, is that something that could be accommodated?
1 : if i need delivery on a certain day  is that something that could be accommodated 
2 : if i need delivery on a certain day is that something that could be accommodated 
3 : if i need delivery o

,id,message,label,sentence
0,5396,I made a purchase recently but I have decided ...,request_refund,made purchase recently decided buying need rec...
1,2202,"Cancel my transfer, please.",cancel_transfer,cancel transfer please
2,3768,How do I change my last name?,edit_personal_details,change last name
3,4967,A withdraw I tried to make at the ATM was decl...,declined_cash_withdrawal,withdraw tried make atm declined know
4,5622,I tried to buy something online yesterday and ...,declined_transfer,tried buy something online yesterday told decl...
5,7752,is there a reason i need to verify top up,verify_top_up,reason need verify top
6,8662,I transferred some funds but I don't see it,balance_not_updated_after_bank_transfer,transferred funds see
7,1362,I need to use my disposable card multiple time...,disposable_card_limits,need use disposable card multiple times day cu...
8,6306,A transaction shows duplicate times.,transaction_charged_twice,transaction shows duplicate times
9,1029,How quickly can you get me my card?,card_delivery_estimate,quickly get card


In [22]:
def clean_data(df):
  # clean stop words 

    def unicode_to_ascii(s):
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
# .decode('UTF-8')
    def clean_stopwords_shortwords(w):
        stopwords_list=set(stopwords.words('english'))
        words = w.split() 
        clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]

        return " ".join(clean_words) 
    def preprocess_sentence(w):
        w = unicode_to_ascii(w.lower().strip())
        w = re.sub(r"([?.!,¿])", r" ", w)
        w = re.sub(r'[" "]+', " ", w)
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
        w=clean_stopwords_shortwords(w)
        w=re.sub(r'@\w+', '',w)
        return w
    df['sentence']=df['message'].map(preprocess_sentence)
    return df

In [23]:
train_ = clean_data(train)
test_ = clean_data(test)

In [24]:
len(train),len(test)

(10466, 2617)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [26]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [27]:
num_classes = np.max(y_train) + 1

In [28]:
num_classes

77

In [29]:
y_train

array([56, 72, 44, ..., 54, 12, 48])

In [30]:
y_test

array([76, 76, 52, ..., 26, 63, 54])

In [31]:
X_train.head(10)

6998    Should I expect to be charged for topping up b...
8659    My Virtual card was rejected for a car rental....
1166                    how do I ask for a physical card?
6900                  need to know where money comes from
9338        Someone stole my phone, what is my next step?
8940                           What does a transfer cost?
2888    Can you look at my account ans see why my card...
9212    I'm pretty sure my top up failed. How do I fix...
8855             What countries do your services support?
9448    Can you tell me if a credit/refund is coming f...
Name: message, dtype: object

In [32]:
X_test.head()

3658    I made a withdraw from the ATM this past holid...
7886    The rate applied to my foreign currency at an ...
2876                         Do you have a return policy?
3822    I'd prefer to use my American Express when put...
2573    I am traveling to Germany, Will I be able to u...
Name: message, dtype: object

# Build Model

## SVM

In [33]:
vectorizer = CountVectorizer(min_df=.01, max_df=.8, ngram_range=[1,1], max_features=300, stop_words='english')

In [34]:
vectorizer

CountVectorizer(max_df=0.8, max_features=300, min_df=0.01, ngram_range=[1, 1],
                stop_words='english')

In [35]:
svm = Pipeline([('vect', vectorizer),
               ('tfidf',TfidfTransformer()),
               ('svm_clf', SGDClassifier(loss='hinge', penalty='l2',alpha=0.003,n_jobs=-1,random_state=0,max_iter=20))])
svm.fit(X_train,y_train)      

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.8, max_features=300, min_df=0.01,
                                 ngram_range=[1, 1], stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('svm_clf',
                 SGDClassifier(alpha=0.003, max_iter=20, n_jobs=-1,
                               random_state=0))])

In [36]:
y_pred_svm = svm.predict(X_test)
y_pred_svm
print(accuracy_score(y_test,y_pred_svm))

0.49474689589302767


In [37]:
for i,j in zip(y_train,train['label']):
    print(f"{i}:{j}")

Streaming output truncated to the last 5000 lines.
76:reverted_card_payment?
74:pending_top_up
16:wrong_exchange_rate_for_cash_withdrawal
49:supported_cards_and_currencies
51:edit_personal_details
56:exchange_via_app
64:Refund_not_showing_up
9:top_up_limits
20:fiat_currency_support
41:change_pin
8:pending_transfer
64:passcode_forgotten
1:transfer_not_received_by_recipient
40:card_payment_wrong_exchange_rate
32:transfer_not_received_by_recipient
76:card_payment_not_recognised
35:order_physical_card
9:card_payment_wrong_exchange_rate
24:reverted_card_payment?
14:cash_withdrawal_not_recognised
27:card_payment_not_recognised
59:pin_blocked
22:edit_personal_details
62:transfer_into_account
8:order_physical_card
70:card_arrival
11:card_payment_fee_charged
54:request_refund
39:failed_transfer
53:card_payment_fee_charged
60:atm_support
8:change_pin
76:supported_cards_and_currencies
33:wrong_exchange_rate_for_cash_withdrawal
4:wrong_exchange_rate_for_cash_withdrawal
75:getting_virtual_card
47:r

In [38]:
y_pred_svm

array([75, 18, 73, ..., 26, 46, 15])

In [39]:
print(accuracy_score(y_test,y_pred_svm))

0.49474689589302767


In [40]:
%%time
y_pred_svm = svm.predict(X_test)
print(accuracy_score(y_test,y_pred_svm))
print(classification_report(y_test,y_pred_svm, target_names=train['label'].unique()))

0.49474689589302767
                                                  precision    recall  f1-score   support

                                  request_refund       0.71      0.96      0.82        47
                                 cancel_transfer       0.24      0.11      0.15        36
                           edit_personal_details       0.33      0.03      0.06        32
                        declined_cash_withdrawal       0.37      0.56      0.44        25
                               declined_transfer       0.40      0.12      0.18        17
                                   verify_top_up       0.50      0.23      0.31        22
         balance_not_updated_after_bank_transfer       0.71      0.52      0.60        42
                          disposable_card_limits       0.63      0.92      0.75        36
                       transaction_charged_twice       0.30      0.17      0.21        36
                          card_delivery_estimate       0.50      0.79      0.61

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NB

In [41]:
nb = Pipeline([('vect', vectorizer ),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.8, max_features=300, min_df=0.01,
                                 ngram_range=[1, 1], stop_words='english')),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [42]:
y_pred = nb.predict(X_test)

In [43]:
y_pred

array([75, 76, 16, ..., 27, 46,  1])

In [44]:
print(confusion_matrix(y_pred,y_test))

[[44  0  0 ...  0  0  0]
 [ 0 21  0 ...  0  0  0]
 [ 0  0  5 ...  1  0  0]
 ...
 [ 0  0  1 ...  6  0  0]
 [ 0  0  0 ...  0 22  2]
 [ 0  0  0 ...  0  0 18]]


In [45]:
print(accuracy_score(y_pred,y_test))

0.5458452722063037


In [46]:
print(classification_report(y_pred, y_test,target_names=train['label'].unique()))


                                                  precision    recall  f1-score   support

                                  request_refund       0.94      0.83      0.88        53
                                 cancel_transfer       0.58      0.18      0.28       116
                           edit_personal_details       0.16      0.50      0.24        10
                        declined_cash_withdrawal       0.56      0.58      0.57        24
                               declined_transfer       0.06      0.33      0.10         3
                                   verify_top_up       0.27      0.33      0.30        18
         balance_not_updated_after_bank_transfer       0.57      0.77      0.66        31
                          disposable_card_limits       0.83      0.68      0.75        44
                       transaction_charged_twice       0.28      0.28      0.28        36
                          card_delivery_estimate       0.83      0.69      0.75        35
         

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## DNN

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
tfidf = TfidfVectorizer()
X_train = (tfidf.fit_transform(train['sentence'])).toarray()


In [49]:
train_size = int(len(train) * 0.7)

x_train = X_train[:train_size]
x_test = X_train[train_size:]


In [50]:
y_train = train['label'][:train_size]
y_test = train['label'][train_size:]

In [51]:
X_train.shape

(10466, 2141)

In [52]:
print(x_train.shape)
print(x_test.shape)

(7326, 2141)
(3140, 2141)


In [53]:
y_train.head()

0              request_refund
1             cancel_transfer
2       edit_personal_details
3    declined_cash_withdrawal
4           declined_transfer
Name: label, dtype: object

In [54]:
y_test.head()

7326                    card_payment_wrong_exchange_rate
7327             wrong_exchange_rate_for_cash_withdrawal
7328                                   declined_transfer
7329    balance_not_updated_after_cheque_or_cash_deposit
7330                            virtual_card_not_working
Name: label, dtype: object

In [55]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [56]:
token = text.Tokenizer(num_words=1000,char_level=False)

In [57]:
token

In [58]:
y_train = utils.np_utils.to_categorical(y_train, num_classes)
y_test = utils.np_utils.to_categorical(y_test, num_classes)

In [65]:
print(y_train.shape)
print(y_train[2])

(7326, 77)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]


In [60]:
from keras.layers import Activation

In [61]:
model = Sequential()
# model.add(Dense(512,input_shape=(1,), activation='relu'))
model.add(Dense(512,kernel_initializer = 'uniform', activation='relu', input_dim = 2141))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))    
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

%time
epochs = 5
batch_size = 64
# history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               1096704   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 77)                39501     
                                                                 
 activation (Activation)     (None, 77)                0         
                                                                 
Total params: 1,398,861
Trainable params: 1,398,861
Non-

In [62]:
score = model.evaluate(x_test, y_test,
                       batch_size=32, verbose=1)
print('Test accuracy:', score[1])

99/99 [==============================] - 0s 3ms/step - loss: 0.5261 - accuracy: 0.8545
Test accuracy: 0.8544585704803467
